In [13]:
library(tidyverse)

results_df <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

read_ssn <- function(year) {
    ssn_start <- year
    ssn_end <- substr(as.character(year + 1), 3, 4)

    file_path = paste0("./csv/", ssn_start, "-", ssn_end, ".csv")
    read_csv(file_path, show_col_types = FALSE) %>%
    mutate(
        season = paste0(ssn_start, "/", ssn_end),
        game_no = row_number()
    ) %>%
    select(-starts_with("..."))
}

s90_91 <- read_ssn(1990)
s91_92 <- read_ssn(1991)
s92_93 <- read_ssn(1992)
s93_94 <- read_ssn(1993)
s94_95 <- read_ssn(1994)
s95_96 <- read_ssn(1995)

In [14]:
df_to_long <- function(ssn_df) {
    results_df %>%
    select(
        season,
        ssn_game_no,
        game_date,
        competition,
        venue,
        opposition,
        outcome,
        score,
        attendance
    ) %>%
    rename(
        game_no = ssn_game_no
    ) %>%
    inner_join(
        ssn_df,
        by = c("season", "game_no")
    ) %>%
    arrange(
        game_date
    ) %>%
        mutate(
        across(-game_no, as.character)
    ) %>%
    select(
        -game_date,
        -venue,
        -opposition,
        -outcome,
        -score,
        -attendance
    ) %>%
    pivot_longer(
        cols = -c("season", "competition", "game_no"),
        names_to = "player_name",
        values_to = "appearance"
    ) %>%
    filter(
        !is.na(appearance)
    ) %>%
    mutate(
        off_for = case_when(
            str_detect(appearance, "\\*") ~ "12",
            str_detect(appearance, "#") ~ "13",
            str_detect(appearance, "\\+") ~ "14",
            .default = NA_character_
        ),
        off_for = as.numeric(off_for),
        shirt_no = str_remove_all(appearance, "\\*|\\+|\\#"),
        shirt_no = as.numeric(shirt_no),
        role = ifelse(shirt_no <= 11, "starter", "sub")
    ) %>%
    arrange(
        game_no,
        shirt_no
    )
}

s90_91_long <- df_to_long(s90_91)
s91_92_long <- df_to_long(s91_92)
s92_93_long <- df_to_long(s92_93)
s93_94_long <- df_to_long(s93_94)
s94_95_long <- df_to_long(s94_95)
s95_96_long <- df_to_long(s95_96)

long_ssns <- rbind(s90_91_long, s91_92_long, s92_93_long, s93_94_long, s94_95_long, s95_96_long)

In [15]:
final_ssn_df <- long_ssns %>%
    left_join(
        long_ssns %>%
        filter(
            !is.na(off_for) | role == "sub"
        )  %>%
        select(
            season,
            game_no,
            shirt_no,
            off_for
        ),
    by = c(
        "season" = "season",
        "game_no" = "game_no",
        "shirt_no" = "off_for")
    ) %>%
    rename(
        on_for = shirt_no.y
    ) %>%
    select(
        season,
        game_no,
        player_name,
        shirt_no,
        role,
        on_for,
        off_for
    )

In [16]:
write_csv(final_ssn_df, "./output/apps_long.csv")